<a href="https://colab.research.google.com/github/mzucali/pyMinColab/blob/main/ExtractFromSummaryEMPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEGGE IL FILE SUMMARY.TXT ed estrae DL e SD:
>input=summary.txt


In [ ]:
# load and read summary file from EMPA
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
file_in = fn

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 14 11:30:45 2023
GRAZIE ALESSANDRO COMUNIAN
@author: alex
"""

#file_in = open("summary.txt", "r")
outFile = open("result2.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    line = line.replace("\t\t","\t")
    line = line.replace("  ","")
    line_s = line.split()
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                comm = line_s[6]
                # Just a spacing line
                print()
        except IndexError:
            pass
        
        if(line_s[0] == "Curr.(A)"):
#        if(line_s[0] == "Element	"):  
            print("ID {0} Comment".format(lines[i+1][:-1]))

            outFile.write("ID {0} Comment\n".format(lines[i+1][:-1]))
            for j in range(2,12):
                print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
                outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
                #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt



In [ ]:
newopen = open("result3.txt", "w")
with open("result2.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  print(first_line)

with open("result2.txt") as f:
  for line in f:
    if 'ID Element	Peak(mm)' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'  
      newopen.write(line)     
newopen.close()

### IL FILE result3 HA ANCORA SPAZI NON COERENTI
#31 marzo >> sistemato

RESULT3.TXT con dati estratti di SD e DL

In [ ]:
file_in2 = open("summary.txt", "r")
lines = file_in2.readlines()
names = []
Xs = []
Ys = []

for i, line in enumerate(lines):
    line_s = line.split()
    #print(line_s)
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                name = line_s[6]
                names.append(name)
                # Just a spacing line
                #print(name) 
            elif(line_s[0] == "Stage"):
                # Keep track of the comment value
                x = line_s[3]
                Xs.append(x)
                y = line_s[5]
                Ys.append(y)
                # Just a spacing line
                #print(x)
        except IndexError:
            pass

import pandas as pd

print("Sample: ",len(names))
print("X: ", len(Xs))
print("Y: ", len(Ys))
out = pd.DataFrame(
    {'Sample': names, 
     'X':Xs, 
     'Y':Ys
     })
#list(zip(names, Xs, Ys)))
print(out)
out.to_csv('coordinate.csv', index=False)

COORDINATE.CSV con sample name e coordinate X e Y

TUTTO DA **SCARICARE**